In [ ]:
!pip install streamlit

     |████████████████████████████████| 9.1 MB 3.8 MB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
     |████████████████████████████████| 178 kB 33.2 MB/s 
     |████████████████████████████████| 180 kB 37.3 MB/s 
     |████████████████████████████████| 111 kB 43.2 MB/s 
     |████████████████████████████████| 4.3 MB 38.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 126 kB 70.5 MB/s 
     |████████████████████████████████| 791 kB 20.0 MB/s 
     |████████████████████████████████| 374 kB 25.4 MB/s 
  Created wheel for pympler: filename=Pympler-0.9-py3-none-any.whl size=164824 sha256=9ea1239c8ff10e2fe3a0a8ea4fdc842726698ced214bd82ba03310c609c7151c
  Stored in directory: /root/.cache/pip/wheels/1a/f3/d8/35d5614ea4ddd295ffb9372a5f2f9570d9593d1ea4be33ec6d
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=c411c0345b5daa9b076cfdfde01ad7f3c31fd6ebad797c37d51c6408211a0549
  Stored in direc

In [ ]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import io, img_as_float
from skimage.filters import gaussian, median
from skimage import util
from skimage.color import rgb2gray
from google.colab.patches import cv2_imshow
def read_image(sample_image):
  sample_image = cv2.imread(sample_image)
  sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
  im = Image.fromarray(sample_image)
  im.show()
  sample_img_gray = cv2.cvtColor(sample_image,cv2.COLOR_BGR2GRAY)
  #img_gray = Image.fromarray(sample_img_gray)
  #img_gray.show()
  return sample_img_gray,sample_image

def patch_removal(img):
  gray = cv2.cvtColor( img, cv2.COLOR_BGR2GRAY )
  retval, binary = cv2.threshold(gray,190,255,cv2.THRESH_BINARY)
  #cv2_imshow(binary)
  dst = cv2.inpaint(img,binary,5,cv2.INPAINT_TELEA)
  #cv2_imshow(dst)
  return dst

def noise_removal(img):
  img = np.array(img)
  #cv2.imshow("Original",img)

  #gauss_noise = util.random_noise(img,mode="gaussian") # add gaussian noise
  #sp_noise = util.random_noise(gauss_noise,mode="s&p") # add s&p noise to image with gaussian noise
  final_img = img_as_float(rgb2gray(img))
  #cv2.imshow("noise",final_img)

  gauss_rem = gaussian(final_img, sigma=1, mode='constant', cval=0.0)  # to remove gaussian noise
  #after removing gaussian noise
  #cv2.imshow("gaussian_rem",gauss_rem)

  from skimage.morphology import disk  
  #Disk creates a circular structuring element, similar to a mask with specific radius
  sp_rem = median(gauss_rem, disk(3), mode='constant', cval=0.0)
  #after removing salt and pepper noise
  #cv2.imshow("s&p_rem",sp_rem)
  #plt.imshow(sp_rem,cmap = 'gray')
  return sp_rem

def hair_removal(img):
  gray_scale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  # cross shaped kernel
  kernel = cv2.getStructuringElement(1,(15,15))
  # BlackHat transform on the grayscale image to find the hair contours
  blackhat = cv2.morphologyEx(gray_scale, cv2.MORPH_BLACKHAT, kernel)

  # intensifying the hair contours for the inpainting algorithm
  retval, binary = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)

  # inpainting the original image
  dst = cv2.inpaint(img,binary,1,cv2.INPAINT_TELEA)
  '''
  fig = plt.figure(figsize=(15,5))
  ax = fig.add_subplot(122)
  plt.imshow(dst)
  ax.set_title('Hair Removal',fontsize=18)
  '''
  
  return dst

def log_transform(im):
  im_log_law_transformation = 0.7 * (np.log(1 + np.float32(im)))
  ''''
  fig = plt.figure(figsize=(15,5))
  ax = fig.add_subplot(122)
  plt.imshow(im_log_law_transformation,cmap="gray")
  ax.set_title('log Transform',fontsize=18)
  '''
  return im_log_law_transformation

def white_back(sample_img_gray):
  (T, sample_img_gray) = cv2.threshold(sample_img_gray, 0, 255,
	cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  img_white = Image.fromarray(sample_img_gray)
  '''
  fig = plt.figure(figsize=(15,5))
  ax = fig.add_subplot(122)
  plt.imshow(sample_img_gray,cmap="gray")
  ax.set_title('white background',fontsize=18)
  '''
  return sample_img_gray

def ada_morph(sample_img_gray):
  adaptive_thresh = cv2.adaptiveThreshold(sample_img_gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
  kernel = np.ones((5,5), np.uint8)
  img_erosion = cv2.erode(sample_img_gray, kernel, iterations=4)
  ed = 255 - img_erosion
  edge = Image.fromarray(ed)
  edge.show()
  '''
  fig = plt.figure(figsize=(15,5))
  ax = fig.add_subplot(122)
  plt.imshow(img_erosion,cmap="gray")
  ax.set_title('ada_morph',fontsize=18)
  '''
  return ed

def contours(ed,sample_image,j):
  hh, ww = sample_image.shape[:2]
  contours,_ = cv2.findContours(ed,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
  area_m = 0
  for i,c in enumerate(contours):
    
    colour = (0,0,255)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])

    cnt_norm = c - [cx, cy]
    cnt_scaled = cnt_norm * 1.5
    cnt_scaled = cnt_scaled + [cx, cy]
    cnt_scaled = cnt_scaled.astype(np.int32)
    area = cv2.contourArea(c)
    if area>area_m:
      cnt_inc = cnt_scaled
      
      area_m = area
  
  cv2.drawContours(sample_image,[cnt_inc],-1,colour,1)
  
  zero_array  = np.zeros(sample_image.shape[:-1]).astype(np.uint8)
  cv2.fillPoly(zero_array, [cnt_inc], 255)
  sel = zero_array != 255
  sample_image[sel] = [255,255,255]
  img_seg = Image.fromarray(sample_image)
  return img_seg

  #img_seg.save(f'/content/drive/MyDrive/Capstone_seg/segmented_melanoma_images/segmented_mel_extra2_image{j}.png')



In [ ]:
#ltp
import numpy as np
def get_pixel(img,x,y,c):
  new_val = 0
  sliding_window1 = c+5 #35
  sliding_window2 = c-5 #25 c=30
  # c+t c-t 
  try :
    if img[x][y] >= sliding_window2 and img[x][y] <= sliding_window1:
      new_val = 0
    elif img[x][y] > sliding_window1:
      new_val = 1
    elif img[x][y] < sliding_window2:
      new_val = -1
  except :
    pass

  return  new_val

def upper_pat(n):
  if n==-1:
    return str(0)
  else:
    return str(n)

def lower_pat(n):
  if n==-1:
    return '1'
  elif n==1:
    return '0'
  else:
    return str(n)

def get_features(img, img_height, img_width):

  # numpy array with same height and width of img
  pattern_img = np.zeros((img_height, img_width),np.uint8)

  for i in range(0,img_height):
    bin1 = ""
    bin2 = ""
    for j in range(0,img_width):
      center = img[i][j]
      #right
      pat = get_pixel(img,i,j+1,center)
      up = upper_pat(pat)
      bin1 = bin1 + up
      lower = lower_pat(pat)
      bin2 = bin2 + lower

      #top-right
      pat = get_pixel(img,i-1,j+1,center)
      up = upper_pat(pat)
      bin1 = bin1 + up
      lower = lower_pat(pat)
      bin2 = bin2 + lower

      #top
      pat = get_pixel(img,i-1,j,center)
      up = upper_pat(pat)
      bin1 = bin1 + up
      lower = lower_pat(pat)
      bin2 = bin2 + lower

      #top-left
      pat = get_pixel(img,i-1,j-1,center)
      up = upper_pat(pat)
      bin1 = bin1 + up
      lower = lower_pat(pat)
      bin2 = bin2 + lower

      #left
      pat = get_pixel(img,i,j-1,center)
      up = upper_pat(pat)
      bin1 = bin1 + up
      lower = lower_pat(pat)
      bin2 = bin2 + lower
      
      #bottom-left
      pat = get_pixel(img,i+1,j-1,center)
      up = upper_pat(pat)
      bin1 = bin1 + up
      lower = lower_pat(pat)
      bin2 = bin2 + lower

      #bottom
      pat = get_pixel(img,i+1,j,center)
      up = upper_pat(pat)
      bin1 = bin1 + up
      lower = lower_pat(pat)
      bin2 = bin2 + lower

      #bottom-right
      pat = get_pixel(img,i+1,j+1,center)
      up = upper_pat(pat)
      bin1 = bin1 + up
      lower = lower_pat(pat)
      bin2 = bin2 + lower
      
      #print(bin1,"\n")
      #print(bin2,"\n")
      bin1 = bin1[::-1]
      bin2 = bin2[::-1]
      pattern_img[i, j] = int(bin1,2) + int(bin2,2)
      bin1=""
      bin2=""
      
      #print(pattern_img[i, j])
  features = []
  histogram = np.histogram(pattern_img, bins = np.arange(17))[0]
  #print(histogram)
  features.append(histogram)
  features = np.concatenate(features, axis = 0)
  return features

In [ ]:
from skimage.feature import greycomatrix, greycoprops
import glob
from keras.preprocessing import image
import numpy as np
def glcm_feat(img):

  glcm_lis = []
  glcm = greycomatrix(np.array(img),distances=[5], angles=[0],symmetric=True, normed=True)
  d = greycoprops(glcm, 'dissimilarity')[0, 0]
  c = greycoprops(glcm, 'correlation')[0, 0]

  ener = greycoprops(glcm, 'energy')[0, 0]


  glcm_lis.append([d,c,ener])
  return glcm_lis


In [ ]:
%%writefile app.py
import tensorflow as tf
import streamlit as st
import pickle

@st.cache(allow_output_mutation=True)
def load_model():
  model = pickle.load(open('/content/final_model.hdf5','rb')) #enter model here!!
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # Skin Cancer Classification
         """
         )

file = st.file_uploader("Please upload your image here....", type=["jpg", "png"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)

def import_and_predict(image_data, model): 
  
  sample_img_gray,sample_image = read_image(image_data)
  sample_image = hair_removal(sample_image)
  sample_image = patch_removal(sample_image)
  sample_img_gray = noise_removal(sample_image)
  sample_img_gray = cv2.normalize(sample_img_gray, None, 0, 255, cv2.NORM_MINMAX, dtype = cv2.CV_8U)

  sample_img_gray = log_transform(sample_img_gray)
  sample_img_gray = cv2.normalize(sample_img_gray, None, 0, 255, cv2.NORM_MINMAX, dtype = cv2.CV_8U)


  sample_img_gray = white_back(sample_img_gray)
  for i in range(len(sample_img_gray)):
    for j in range(len(sample_img_gray[i])):
      if sample_img_gray[i][j]==0:
        if (len(sample_img_gray) - i) <10 or (len(sample_img_gray) - j) < 10:
          sample_img_gray[i][j] = 255

  ed = ada_morph(sample_img_gray)
  ret_image = contours(ed,sample_image)  # preprocessed and segmented image

  #ltp
  import cv2
  img = cv2.cvtColor(ret_image, cv2.COLOR_BGR2GRAY)
  #img = image.img_to_array(img, dtype='uint8')
  lis = []
  lis.append(get_features(img,img.shape[0],img.shape[1]))
  lis1 = lis/np.linalg.norm(lis)
  lis = list(lis1[0])
  glcm_lis = glcm_feat(img)
  lis.extend(glcm_lis[0])
  liss = pd.DataFrame([lis],columns= ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19'])


  # output after model pred to be returned
  y_pred = model.predict(liss[['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19']])
  return y_pred # array
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file) #input
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    map_pred = {0:'Melanoma', 1:'NV', 2:'BKL'}
    st.write("The detected skin lesion is "+map_pred[predictions])
#Whatever is our output should be printed rest all needs to be removed
   # score = tf.nn.softmax(predictions[0]) #svm  output
    #st.write(prediction)
    #st.write(score) #svmoutput
    #print(
   # "This image most likely belongs to {} with a {:.2f} percent confidence."
   # .format(class_names[np.argmax(score)], 100 * np.max(score))
#) #depends on our output

Overwriting app.py


In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 16.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=7dcdeb1a5bf575541264867041a635b7253faed38b491c2267ddf01a5fa93bd9
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!ngrok authtoken #Insert Authentication Token here, obtained from Ngrok

NAME:
   authtoken - save authtoken to configuration file

USAGE:
   ngrok authtoken [command options] [arguments...]

DESCRIPTION:
   The authtoken command modifies your configuration file to include
   the specified authtoken. By default, this configuration file is located
   at $HOME/.ngrok2/ngrok.yml

   The ngrok.com service requires that you sign up for an account to use
   many advanced service features. In order to associate your client with
   an account, it must pass a secret token to the ngrok.com service when it
   starts up. Instead of passing this authtoken on every invocation, you may
   use this command to save it into your configuration file so that your
   client always authenticates you properly.

EXAMPLE:
    ngrok authtoken BDZIXnhJt2HNWLXyQ5PM_qCaBq0W2sNFcCa0rfTZd

OPTIONS:
   --config 		save in this config file, default: ~/.ngrok2/ngrok.yml
   --log "false"	path to log file, 'stdout', 'stderr' or 'false'
   --log-format "term"	log record format: 'term', 'logfmt',

In [ ]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok
url=ngrok.connect(port=8501)
url

<NgrokTunnel: "http://8578-34-90-221-232.ngrok.io" -> "http://localhost:80">

In [ ]:
!cat /content/nohup.out

Alternate if  ngrok command not work


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-12-11 09:38:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  11.9MB/s    in 1.1s    

2021-12-11 09:38:44 (11.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
 !curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url 
  

/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
!streamlit run /content/app.py  # add our model


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.221.255.5:8501

  Stopping...
  Stopping...
